# Equipo #1

- Contreras Molina Vannesa Samanda
- Cruz Cortés Alba Cristina
- Delgado Barajas Fabiola Alejandra
- Hernandez Martinez Abraham
- Huerta Rangel Alejandra
- Rincon Morales David

# Lectura de BD


In [1]:
import pandas as pd
import numpy as np

import json as js
from sqlalchemy import create_engine
import mysql.connector

import datetime as dt

In [ ]:
!pip install pymysql

In [2]:
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
"""
Change to a function try-catch
"""

mydb = mysql.connector.connect(
  host="relational.fit.cvut.cz",
  port = 3306,
  db = "financial",
  user="guest",
  passwd="relational"
)

In [4]:
account = pd.read_sql(con = mydb, sql = "select account_id, date as account_creation_date, district_id as account_location from account")
client = pd.read_sql(con = mydb, sql="Select client_id, birth_date, gender From client")
trans = pd.read_sql(con = mydb, sql = "select account_id,trans_id, date as trans_date, operation, amount as trans_amount, balance as balance_after_trans from trans")
disp = pd.read_sql(con = mydb, sql = "select client_id, account_id, type from disp")
district = pd.read_sql(con = mydb, sql = "select district_id as account_location, A2 as nombre_distrito, A3 as region_distrito from district")

In [5]:
disposition = disp.merge(client, on = "client_id", how = "inner")

In [6]:
account_new = account.merge(disposition, on = "account_id", how = "inner")

In [7]:
transactions = trans.merge(account_new, on = "account_id", how = "inner")

transactions = transactions.merge(district, on = "account_location", how = "inner")

transactions.drop(["account_location", "client_id"], axis=1, inplace=True)

transactions = transactions.reset_index(drop=True)

In [8]:
transactions.isnull().sum()

account_id                    0
trans_id                      0
trans_date                    0
operation                218311
trans_amount                  0
balance_after_trans           0
account_creation_date         0
type                          0
birth_date                    0
gender                        0
nombre_distrito               0
region_distrito               0
dtype: int64

In [9]:
transactions["operation"].value_counts()

VYBER             517031
PREVOD NA UCET    254449
VKLAD             181962
PREVOD Z UCTU      81601
VYBER KARTOU        9271
Name: operation, dtype: int64

In [10]:
transactions["operation"].fillna(value="Operacion no definida", inplace=True)
transactions.isnull().sum()

account_id               0
trans_id                 0
trans_date               0
operation                0
trans_amount             0
balance_after_trans      0
account_creation_date    0
type                     0
birth_date               0
gender                   0
nombre_distrito          0
region_distrito          0
dtype: int64

In [11]:
transactions["operation"].value_counts()

VYBER                    517031
PREVOD NA UCET           254449
Operacion no definida    218311
VKLAD                    181962
PREVOD Z UCTU             81601
VYBER KARTOU               9271
Name: operation, dtype: int64

In [12]:
def columns_to_datetime(df):
    
    df["trans_date"] = pd.to_datetime(df["trans_date"])
    df["account_creation_date"] = pd.to_datetime(df["account_creation_date"])
    df["birth_date"] = pd.to_datetime(df["birth_date"])
    
    df["transaction_month"] = df["trans_date"].dt.month
    df["transaction_year"] = df["trans_date"].dt.year
    
    df.drop(["trans_date"], axis=1, inplace=True)
    
    return df

In [13]:
def get_age(df):
    """
    Gets years from date.
    
    Need to update function in order to allow any column as parameter
    """
    
    now = pd.datetime.now()
    
    transactions['account_creation_date'] = transactions['account_creation_date'].where(transactions['account_creation_date'] < now, transactions['account_creation_date'] -  np.timedelta64(100, 'Y'))   # 2
    transactions['account_age'] = (now - transactions['account_creation_date']).astype('<m8[Y]')
    transactions['account_age'] = transactions['account_age'].astype(int)
    
    transactions['birth_date'] = transactions['birth_date'].where(transactions['birth_date'] < now, transactions['birth_date'] -  np.timedelta64(100, 'Y'))   # 2
    transactions['client_age'] = (now - transactions['birth_date']).astype('<m8[Y]')
    transactions['client_age'] = transactions['client_age'].astype(int)
    
    transactions.drop(["account_creation_date"], axis=1, inplace=True)
    transactions.drop(["birth_date"], axis=1, inplace=True)
    
    return df

In [14]:
transactions = columns_to_datetime(transactions)

In [15]:
transactions = get_age(transactions)

/home/david/Documents/Python envs/mineria/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime instead.
  


In [16]:
transactions

,account_id,trans_id,operation,trans_amount,balance_after_trans,type,gender,nombre_distrito,region_distrito,transaction_month,transaction_year,account_age,client_age
0,1,1,VKLAD,1000,1000,OWNER,F,Pisek,south Bohemia,3,1995,26,50
1,1,5,PREVOD Z UCTU,3679,4679,OWNER,F,Pisek,south Bohemia,4,1995,26,50
2,1,6,PREVOD Z UCTU,3679,20977,OWNER,F,Pisek,south Bohemia,5,1995,26,50
3,1,7,PREVOD Z UCTU,3679,26835,OWNER,F,Pisek,south Bohemia,6,1995,26,50
4,1,8,PREVOD Z UCTU,3679,30415,OWNER,F,Pisek,south Bohemia,7,1995,26,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1262620,11244,3528724,Operacion no definida,164,34017,OWNER,F,Decin,north Bohemia,8,1998,24,72
1262621,11244,3528725,Operacion no definida,158,46507,OWNER,F,Decin,north Bohemia,9,1998,24,72
1262622,11244,3528726,Operacion no definida,121,22426,OWNER,F,Decin,north Bohemia,10,1998,24,72
1262623,11244,3528727,Operacion no definida,63,11083,OWNER,F,Decin,north Bohemia,11,1998,24,72


In [17]:
sample = transactions.sample(frac=0.3)

In [18]:
sample.to_csv("transactions.csv", index=False)

In [20]:
with open("cred_etl.json", "r+") as file:
    cred = js.load(file)

In [21]:
db =  cred['db']
host =  cred['host']
port =  cred['port']
flavour = cred['flavour']
username = cred['username']
password = cred['password']

In [22]:
etl_jdbc = f"{flavour}://{username}:{password}@{host}:{port}/{db}"
etl_conn = create_engine(etl_jdbc).connect()

In [23]:
sample.to_sql("DataEquipo1", con=etl_conn, if_exists="replace", index=False, chunksize=10000)